# Summarize skill results
***

**Author**: Chus Casado Rodríguez<br>
**Date**: 16-06-2023<br>


**Introduction**:<br>
This notebook creates a table comparing the skill of the diverse notification criteria optimized for different f-scores.

In [1]:
import os
path_root = os.getcwd()
import glob
import numpy as np
import pandas as pd
import xarray as xr
from datetime import datetime, timedelta
import pickle
import yaml

import warnings
warnings.filterwarnings("ignore")

os.chdir('../py/')
from compute import hits2skill
os.chdir(path_root)

## 1 Configuration

In [2]:
with open("../conf/config.yml", "r", encoding='utf8') as ymlfile:
    cfg = yaml.load(ymlfile, Loader=yaml.FullLoader)

### 1.1 Reporting points

In [3]:
# area threshold
area_threshold = cfg.get('reporting_points', {}).get('area', 500)

# reporting points
path_stations = cfg.get('reporting_points', {}).get('output', '../results/reporting_points/')
file_stations = f'{path_stations}reporting_points_over_{area_threshold}km2.parquet'

# catchments
catchments = cfg.get('reporting_points', {}).get('catchments', None)

# minimum performance required from the reporting points
min_kge = cfg.get('reporting_points', {}).get('KGE', None)

### 1.2 Hits

In [4]:
# parameters of the rolling window used to compute hits
window = cfg.get('hits', {}).get('window', 1)

# dissagregate the analysis by seasons?
seasonality = cfg.get('hits', {}).get('seasonality', False)

# path that contains the NetCDFs with hit, misses and false alarms pro
path_in = cfg.get('hits', {}).get('output', '../results/hits/')
path_in = f'{path_in}window_{window}/'

### 1.3 Skill

In [5]:
# current operationa criteria
current_criteria = cfg.get('skill', {}).get('current_criteria', None)

# fixed notification criteria
min_leadtime = cfg.get('skill', {}).get('leadtime', 60) 
min_area = cfg.get('skill', {}).get('area', 2000) 

# coefficient of the fbeta-score
betas = [0.8, 1, 1.2]

# path where results will be saved
path_out = cfg.get('skill', {}).get('output', f'../results/skill/')
if min_kge is not None:
    path_out = f'{path_out}window_{window}/kge_{min_kge}/'
else:
    path_out = f'{path_out}window_{window}/no_kge/'

## 2 Data

### 2.1 Reporting points

I load all the stations that where selected in a previous [notebook](3_0_select_stations.ipynb).

In [6]:
# load table of fixed reporting points
stations = pd.read_parquet(file_stations)
stations[['X', 'Y', 'area']] = stations[['X', 'Y', 'area']].astype(int)

# select stations that belong to the selected catchments
if catchments is not None:
    if isinstance(catchments, list) is False:
        catchments = [catchments]
    stations = stations.loc[stations.catchment.isin(catchments),:]

# remove points with a performance (KGE) lower than the established threshold
if min_kge is not None:
    mask_kge = ~(stations.KGE <= min_kge)
    stations = stations.loc[mask_kge]
else:
    # remove station with erroneous behaviour
    stations = stations.loc[~(stations.n_events_obs >= 6)]

# mask stations with events
stations_w_events = (stations.n_events_obs > 0)

print('All points')
print('----------')
print(f'no. reporting points:\t\t{stations.shape[0]}')
print('no. stations with events:\t{0}'.format(stations_w_events.sum()))
print('no. observed events:\t\t{0}'.format(stations.n_events_obs.sum()))

# select stations according to catchment area
if min_area > area_threshold:
    stations_optimize = stations.loc[stations.area >= min_area].index
else:
    stations_optimize = stations.index

print('\nPoints selected for otimization')
print('-------------------------------')
print(f'no. reporting points:\t\t{len(stations_optimize)}')
print('no. stations with events:\t{0}'.format((stations.loc[stations_optimize, 'n_events_obs'] > 0).sum()))
print('no. observed events:\t\t{0}'.format(stations.loc[stations_optimize, 'n_events_obs'].sum()))

# suffix that will be used when saving plots
suffix = f'{min_area}km2_{len(stations_optimize)}points'

All points
----------
no. reporting points:		2357
no. stations with events:	980
no. observed events:		1469

Points selected for otimization
-------------------------------
no. reporting points:		1424
no. stations with events:	538
no. observed events:		748


### 2.2 Hits, misses and false alarms

In [7]:
# import hits for each station
hits_stn = xr.open_mfdataset(f'{path_in}*.nc', combine='nested', concat_dim='id')
# reorder persistences
hits_stn = hits_stn.sel(persistence=['1/1', '2/4', '2/3', '2/2', '3/4', '3/3'])
# extract selected stations
hits_stn = hits_stn.sel(id=stations.index.to_list()).compute()

# convert to NaN values at long leadtimes for which the persistence criteria is impossible to meet
hits_stn = hits_stn.astype(float)
for persistence in hits_stn.persistence.data:
    last_leadtime = int(persistence.split('/')[0]) - 1
    if last_leadtime > 0:
        hits_stn.sel(persistence=persistence)[dict(leadtime=slice(-last_leadtime, None))] = np.nan

# subset of the 'hits' dataset with the stations selected for the optimization
hits_opt = hits_stn.sel(id=stations_optimize).sum('id', skipna=False)

## 3 Analysis

In this section I will compute the skill of the EFAS predictions in different ways. In all the following sections I will work with three metrics: $recall$, $precision$ and the $f_{beta}$ score. The three metrics are based in the contingency table of hits ($TP$ for true positives), false alarms ($FP$ for false positives) and misses ($FN$ for false negatives).

$$recall = \frac{TP}{TP + FN}$$
$$precision = \frac{TP}{TP + FP}$$
$$f_{beta} = \frac{(1 + \beta^2) \cdot TP}{(1 + \beta^2) \cdot TP + \beta^2 \cdot FN + FP}$$


### 3.2 Compare approaches
#### 3.2.1 Import optimize criteria

In [8]:
criteria = {'current': current_criteria}
for beta in betas:
    metric = f'f{beta}'
    file = glob.glob(f'{path_out}{metric}/*{suffix}.pkl')[0]
    opt_crit = pickle.load(open(file, 'rb'))
    for app, crit in opt_crit.items():
        if metric in crit:
            del crit[metric]
        criteria[f'{metric}_{app}'] = crit

#### 3.2.2 Compare approaches

In [9]:
# transform criteria into a DataFrame
summary_criteria = pd.concat([pd.DataFrame(crtr, index=[i]) for i, crtr in criteria.items()], axis=0)
summary_criteria.approach = [''.join([x[0].upper() for x in app.split('_')]) for app in summary_criteria.approach]
summary_criteria['OF'] = [x.split('_')[0] if x != 'current' else '' for x in summary_criteria.index]
summary_criteria = summary_criteria[['approach', 'OF', 'probability', 'persistence']]

In [10]:
# compute hits, misses and false alarms
summary_hits = pd.DataFrame({i: hits_opt.sel(leadtime=min_leadtime).sel(crtr).to_pandas() for i, crtr in criteria.items()}).transpose()
summary_hits = summary_hits.astype(int)
summary_hits['no_events'] = summary_hits.TP + summary_hits.FN

In [11]:
# compute skill
summary_hits['recall'] = summary_hits.TP / (summary_hits.TP + summary_hits.FN)
summary_hits['precision'] = summary_hits.TP / (summary_hits.TP + summary_hits.FP)
for beta in betas:
    summary_hits[f'f{beta}'] = (1 + beta**2) * summary_hits.TP / ((1 + beta**2) * summary_hits.TP + beta**2 * summary_hits.FN + summary_hits.FP)

In [12]:
# concat criteria, hits and summary data frames
summary = pd.concat((summary_criteria, summary_hits), axis=1)
summary.sort_values('approach', inplace=True)

summary

approach    OF  probability  \
current                                   1D+1P              0.300   
f0.8_1_deterministic_+_1_probabilistic    1D+1P  f0.8        0.575   
f1_1_deterministic_+_1_probabilistic      1D+1P    f1        0.400   
f1.2_1_deterministic_+_1_probabilistic    1D+1P  f1.2        0.400   
f0.8_brier_weighted                          BW  f0.8        0.650   
f1_brier_weighted                            BW    f1        0.450   
f1.2_brier_weighted                          BW  f1.2        0.450   
f0.8_model_mean                              MM  f0.8        0.375   
f1_model_mean                                MM    f1        0.150   
f1.2_model_mean                              MM  f1.2        0.150   
f0.8_member_weighted                         MW  f0.8        0.475   
f1_member_weighted                           MW    f1        0.400   
f1.2_member_weighted                         MW  f1.2        0.250   

                                       persistence   TP   FN   FP  no_events  \
current                                        3/3  280  468  182        748   
f0.8_1_deterministic_+_1_probabilistic         1/1  312  436  190        748   
f1_1_deterministic_+_1_probabilistic           1/1  368  380  332        748   
f1.2_1_deterministic_+_1_probabilistic         1/1  368  380  332        748   
f0.8_brier_weighted                            1/1  276  472  136        748   
f1_brier_weighted                              1/1  367  381  394        748   
f1.2_brier_weighted                            1/1  367  381  394        748   
f0.8_model_mean                                3/3  277  471  197        748   
f1_model_mean                                  3/3  380  368  453        748   
f1.2_model_mean                                3/3  380  368  453        748   
f0.8_member_weighted                           1/1  298  450  169        748   
f1_member_weighted                             1/1  320  428  222        748   
f1.2_member_weighted                           1/1  394  354  496        748   

                                          recall  precision      f0.8  \
current                                 0.374332   0.606061  0.488137   
f0.8_1_deterministic_+_1_probabilistic  0.417112   0.621514  0.521739   
f1_1_deterministic_+_1_probabilistic    0.491979   0.525714  0.512013   
f1.2_1_deterministic_+_1_probabilistic  0.491979   0.525714  0.512013   
f0.8_brier_weighted                     0.368984   0.669903  0.508173   
f1_brier_weighted                       0.490642   0.482260  0.485497   
f1.2_brier_weighted                     0.490642   0.482260  0.485497   
f0.8_model_mean                         0.370321   0.584388  0.476824   
f1_model_mean                           0.508021   0.456182  0.475101   
f1.2_model_mean                         0.508021   0.456182  0.475101   
f0.8_member_weighted                    0.398396   0.638116  0.516770   
f1_member_weighted                      0.427807   0.590406  0.514147   
f1.2_member_weighted                    0.526738   0.442697  0.472091   

                                              f1      f1.2  
current                                 0.462810  0.443890  
f0.8_1_deterministic_+_1_probabilistic  0.499200  0.482091  
f1_1_deterministic_+_1_probabilistic    0.508287  0.505267  
f1.2_1_deterministic_+_1_probabilistic  0.508287  0.505267  
f0.8_brier_weighted                     0.475862  0.452240  
f1_brier_weighted                       0.486415  0.487172  
f1.2_brier_weighted                     0.486415  0.487172  
f0.8_model_mean                         0.453355  0.435737  
f1_model_mean                           0.480708  0.485415  
f1.2_model_mean                         0.480708  0.485415  
f0.8_member_weighted                    0.490535  0.470896  
f1_member_weighted                      0.496124  0.482237  
f1.2_member_weighted                    0.481074  0.488714

In [13]:
# export
summary.to_csv(f'{path_out}skill_by_criteria.csv', float_format='%.3f', index=False)

***

```Python
hits_stn

hits_1D1P = hits_stn.sel(approach='1_deterministic_+_1_probabilistic',
                         probability=0.375,
                         persistence='1/1',
                         leadtime=min_leadtime)

hits_BW = hits_stn.sel(approach='brier_weighted',
                       probability=0.375,
                       persistence='1/1',
                         leadtime=min_leadtime)

hits_diff = hits_BW - hits_1D1P

tp = hits_diff['TP'].to_pandas()

tp[tp == 0]

tp[tp < 0]

stations[tp > 0].value_counts('river')

stations[tp > 0].value_counts('catchment')

stations[tp > 0][stations.catchment == 'Rhine'].sort_values(['catchment', 'subcatchment', 'river'])

```